In [1]:
from truncatedMVN import TruncMVN
import numpy as np


In [2]:
mean = np.random.randn(4000)

cov = np.random.randn(4000,4000)*0.02
cov = cov@cov.T+np.eye(4000)*0.13

In [3]:
np.linalg.det(cov)

26.358852871136207

In [4]:
planes = np.random.randn(2800,4000)+np.ones([2800,4000])

In [5]:
tm = TruncMVN(mean,cov,planes)

In [6]:
%timeit -n 1 -r 1 tm.sample(samples=40) #before just one of these samples took over 2 minutes :)

299/300
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!1min 26s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [7]:
%timeit -n 1 -r 1 tm.sample(samples=40) #once caching is done, it is faster.

!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!55.9 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [13]:
%timeit -n 1 -r 1 tm.sample(samples=40,usecaching=False) #without caching this is slower either way!

2min 15s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
